In [ ]:
!pip install transformers datasets torch scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Roman Urdu DataSet.csv to Roman Urdu DataSet.csv


In [ ]:
import pandas as pd

# Load your Roman Urdu dataset
df = pd.read_csv("Roman Urdu DataSet.csv")

# Print the column names
print("Your dataset columns are:")
print(df.columns)


Your dataset columns are:
Index(['Review', 'Sentiment', 'Unnamed: 2'], dtype='object')


In [ ]:
import pandas as pd

df = pd.read_csv("Roman Urdu DataSet.csv")

# Display all column names properly
print("Actual column names in the dataset:")
for i, col in enumerate(df.columns):
    print(f"{i+1}. '{col}'")


Actual column names in the dataset:
1. 'Review'
2. 'Sentiment'
3. 'Unnamed: 2'


In [ ]:
# Load the CSV
df = pd.read_csv("Roman Urdu DataSet.csv")

# Show the actual column names
print("Column names:", df.columns.tolist())


Column names: ['Review', 'Sentiment', 'Unnamed: 2']


In [ ]:
# Remove extra spaces from column names
df.columns = df.columns.str.strip()

# Rename for simplicity (optional but clean)
df.rename(columns={'RomanUrduText': 'text', 'Sentiment': 'sentiment'}, inplace=True)


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv("Roman Urdu DataSet.csv")

# Print column names to debug
print("Original column names:", df.columns.tolist())

# Strip spaces from column names
df.columns = df.columns.str.strip()

# Rename if needed (you can adjust based on the actual names printed above)
df.rename(columns={'RomanUrduText': 'text', 'Sentiment': 'sentiment'}, inplace=True)

# Print again to confirm
print("After cleanup:", df.columns.tolist())

# Check a few rows
print(df.head())


Original column names: ['Review', 'Sentiment', 'Unnamed: 2']
After cleanup: ['Review', 'sentiment', 'Unnamed: 2']
                 Review sentiment Unnamed: 2
0             sahi bt h  Positive        NaN
1           Kya bt hai,  Positive        NaN
2            Wah je wah  Positive        NaN
3  Are wha kaya bat hai  Positive        NaN
4    Wah kya baat likhi  Positive        NaN


In [ ]:
# Load the dataset
df = pd.read_csv("Roman Urdu DataSet.csv")

# Show column names to debug issues
print("Original Columns:", df.columns.tolist())

# Strip extra spaces and lowercase the column names
df.columns = df.columns.str.strip().str.lower()
print("Cleaned Columns:", df.columns.tolist())

# Rename columns based on cleaned names
# Example: If 'romanurdu text' and 'sentiment' are the actual names
df.rename(columns={'romanurdu text': 'text', 'sentiment': 'sentiment'}, inplace=True)

# Map sentiment to numerical labels
label_mapping = {'positive': 0, 'neutral': 1, 'negative': 2}
df['label'] = df['sentiment'].str.strip().map(label_mapping)

# Drop missing rows
df.dropna(subset=['text', 'label'], inplace=True)

# Check if everything looks correct
print(df.head())


Original Columns: ['Review', 'Sentiment', 'Unnamed: 2']
Cleaned Columns: ['review', 'sentiment', 'unnamed: 2']


KeyError: ['text']

In [ ]:
# Import necessary libraries
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load dataset
df = pd.read_csv("uber_reviews_without_reviewid.csv")

# Check column names
print("Dataset Columns:", df.columns)

# Convert numerical scores to sentiment labels
def map_sentiment(score):
    if score >= 4:
        return 2  # Positive
    elif score == 3:
        return 1  # Neutral
    else:
        return 0  # Negative

df['label'] = df['score'].apply(map_sentiment)

# Preprocess text (convert to lowercase & remove extra spaces)
def preprocess_text(text):
    return str(text).lower().strip()

df['cleaned_text'] = df['content'].apply(preprocess_text)

# Split dataset into training (80%) and testing (20%)
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['cleaned_text'], df['label'], test_size=0.2, random_state=42
)

# Convert into Hugging Face Dataset format
train_data = Dataset.from_dict({"text": train_texts.tolist(), "label": train_labels.tolist()})
test_data = Dataset.from_dict({"text": test_texts.tolist(), "label": test_labels.tolist()})

print(f"Training Data Size: {len(train_data)}, Testing Data Size: {len(test_data)}")


Using device: cpu
Dataset Columns: Index(['userName', 'userImage', 'content', 'score', 'thumbsUpCount',
       'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt',
       'appVersion'],
      dtype='object')
Training Data Size: 9600, Testing Data Size: 2400


In [ ]:
# Load BERT tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3).to(device)

# Tokenize text data
def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

train_data = train_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

# Remove unnecessary columns
train_data = train_data.remove_columns(["text"])
test_data = test_data.remove_columns(["text"])

# Convert datasets to PyTorch format
train_data.set_format("torch")
test_data.set_format("torch")

# Create a Data Collator to handle batch padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

In [ ]:
# Define evaluation metrics
def compute_metrics(pred):
    logits, labels = pred
    predictions = torch.tensor(logits).argmax(dim=-1)  # Convert to tensor before argmax
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Set Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"  # ✅ Disables W&B logging
)


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.280200,0.234307,0.933750,0.905412,0.933750,0.918811
2,0.143100,0.239801,0.935417,0.939233,0.935417,0.921347


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=1200, training_loss=0.2373023387293021, metrics={'train_runtime': 24951.8453, 'train_samples_per_second': 0.769, 'train_steps_per_second': 0.048, 'total_flos': 1262944405094400.0, 'train_loss': 0.2373023387293021, 'epoch': 2.0})

In [ ]:
# Evaluate model performance
results = trainer.evaluate()
print("Evaluation Results:", results)


Evaluation Results: {'eval_loss': 0.23430658876895905, 'eval_accuracy': 0.93375, 'eval_precision': 0.9054119481734147, 'eval_recall': 0.93375, 'eval_f1': 0.9188112052730696, 'eval_runtime': 906.2441, 'eval_samples_per_second': 2.648, 'eval_steps_per_second': 0.166, 'epoch': 2.0}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Save model and tokenizer for future use
model.save_pretrained("./fine_tuned_bert")
tokenizer.save_pretrained("./fine_tuned_bert")

print("Model saved successfully.")


NameError: name 'model' is not defined

In [ ]:
from transformers import pipeline

# Load fine-tuned model
model_path = "./fine_tuned_bert"
sentiment_pipeline = pipeline("text-classification", model=model_path, tokenizer=model_path)

# Example: Predict sentiment of a university bus review
review = "The bus was late, and the driver was rude."
prediction = sentiment_pipeline(review)
print(prediction)  # Output: [{'label': 'LABEL_0', 'score': 0.98}]


Device set to use cpu


[{'label': 'LABEL_0', 'score': 0.9442834258079529}]


In [ ]:
# Map model output to sentiment labels
label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

# Get prediction
predicted_label = prediction[0]['label']
predicted_sentiment = label_map[int(predicted_label.split("_")[-1])]  # Convert label to readable text
print(f"Predicted Sentiment: {predicted_sentiment}")


Predicted Sentiment: Negative


In [ ]:
from transformers import pipeline

# Load the fine-tuned model
model_path = "./fine_tuned_bert"  # Make sure this is the correct path
sentiment_pipeline = pipeline("text-classification", model=model_path, tokenizer=model_path)


Device set to use cpu


In [ ]:
# Example test reviews
reviews = [
    "The bus was late by 30 minutes and the driver was rude.",
    "The ride was smooth, and the driver was very polite.",
    "The bus was crowded, and the ride was just average. Nothing special."
]

# Make predictions
predictions = sentiment_pipeline(reviews)

# Display results
for review, prediction in zip(reviews, predictions):
    print(f"Review: {review}")
    print(f"Predicted Sentiment: {prediction['label']}\n")


Review: The bus was late by 30 minutes and the driver was rude.
Predicted Sentiment: LABEL_0

Review: The ride was smooth, and the driver was very polite.
Predicted Sentiment: LABEL_2

Review: The bus was crowded, and the ride was just average. Nothing special.
Predicted Sentiment: LABEL_0



In [ ]:
# Mapping of model labels to sentiment categories
label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

# Convert labels and print results
for review, prediction in zip(reviews, predictions):
    label_number = int(prediction['label'].split("_")[-1])  # Extract label index
    sentiment = label_map[label_number]  # Convert to readable format

    print(f"Review: {review}")
    print(f"Predicted Sentiment: {sentiment}\n")


Review: The bus was late by 30 minutes and the driver was rude.
Predicted Sentiment: Negative

Review: The ride was smooth, and the driver was very polite.
Predicted Sentiment: Positive

Review: The bus was crowded, and the ride was just average. Nothing special.
Predicted Sentiment: Negative

